# DevelopmentalStage

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Human Developmental Stages](https://github.com/obophenotype/developmental-stage-ontologies/wiki/HsapDv)
2. [Mouse Developmental Stages](https://github.com/obophenotype/developmental-stage-ontologies/wiki/MmusDv)

Here we show how to access and search DevelopmentalStage ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-developmentalstage --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
developmentalstages = lb.DevelopmentalStage.public(organism="human")
developmentalstages

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = developmentalstages.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = developmentalstages.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.organogenesis_stage

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["organogenesis stage"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = developmentalstages.lookup(developmentalstages.ontology_id)

In [ ]:
lookup.hsapdv_0000015

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
developmentalstages.search("organogenesis").head(3)

By default, search also covers synonyms:

In [ ]:
developmentalstages.search("developmental stage").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
developmentalstages.search("developmental stage", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
developmentalstages.search(
    "Prenatal Stage That Starts With Fertilization",
    field=developmentalstages.definition,
).head()

## Standardize DevelopmentalStage identifiers

Let us generate a `DataFrame` that stores a number of DevelopmentalStage identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "blastula stage"
        "Carnegie stage 03"
        "neurula stage"
        "organogenesis stage"
        "This developmentalstage does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = developmentalstages.validate(df_orig.index, developmentalstages.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="DevelopmentalStage").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="hsapdv", version="2020-03-10", organism="human"
).one()
developmentalstages= lb.DevelopmentalStage.public(public_source=public_source)
developmentalstages

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-developmentalstage
!rm -r test-developmentalstage